In [1]:
%matplotlib inline
import sys
import os
sys.path.append("/Users/chernals/cernbox/Work/packages")
#sys.path.append("/Users/chernals/cernbox/Work/packages/georges")
import pandas as pd
import matplotlib.pyplot as plt
import georges
PATH = "/Users/chernals/Dropbox/IBA/Work/IBA-Optics/beamlines"

In [5]:
bdsim = georges.bdsim.BDSim(bdsim='/Users/chernals/bdsim-build/bdsim', path=PATH, context=context)
bdsim.attach(bl_survey)
bdsim.print_input()

BRHO=2.3114; DEGREE=pi/180.0;ESSSTART: marker; Q1E: quadrupole, l=0.295*m; DRIFT_0: drift, l=0.3329965761*m; Q2E: quadrupole, l=0.295*m; DRIFT_1: drift, l=0.1380008662*m; Q3E: quadrupole, l=0.295*m; DRIFT_2: drift, l=0.8282084377*m; B1E_0: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_1: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_2: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_3: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_4: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_5: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_6: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_7: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_8: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_9: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGREE; B1E_10: rbend, l=0.03929968771*m, angle=-0.0004569261296800628/DEGR

In [4]:
brho = georges.physics.momentum_to_brho(georges.physics.energy_to_momentum(177.5))
context = {
        'PARTICLE': 'PROTON',
        'PC': georges.physics.energy_to_momentum(177.5),
        'BETAX'   : 0.0846155,
        'BETAY'   : 0.0846155,
        'ALPHAX': 0.0,
        'ALPHAY': 0.0,
        'DELTAP': 0.0,
        'DPP': 0.5e-2,
        'IQ1E': 2.741/10/brho/0.0325*0.297/0.295,
        'IQ2E': -5.745/10/brho/0.0325*0.297/0.295,
        'IQ3E': 4.079/10/brho/0.0325*0.297/0.295,
        'IQ47E': -3.062/10/brho/0.0325,
        'IQ56E': 2.66/10/brho/0.0325,
        'IQ8E': 3.5484/10/brho/0.0325,
        'IQ9E': -4.0276/10/brho/0.0325,
        'IQ10E': 4.0352/10/brho/0.0325,
        'MOMENTUM_SLITS_OPENING' : 1.015,
        'DIVERGENCE_SLITS_OPENING_X' : 1.015,
        'DIVERGENCE_SLITS_OPENING_Y' : 1.015,
        'N_TRACKING': 5000,
        'EMITX': 14.3e-6,
        'EMITY': 14.3e-6,
    }
m = georges.madx.Madx(madx='/usr/local/bin/madx', path=PATH, context=context)
bl = georges.Beamline(['ess'], path=PATH, prefix='generic', elements='elements')
bl_survey = georges.madx.survey(line=bl, madx=m)
#bl_twiss= georges.madx.twiss(line=bl, madx=m)
#b = georges.Beam(energy=200).from_5d_multigaussian_distribution(5,
#                                                   XRMS=0.01,
#                                                   PXRMS=0.001,
#                                                   YRMS=0.01,
#                                                   PYRMS=0.001,
#                                                   DPPRMS=0.00)
#bl_track = georges.madx.track(line=bl, madx=m, beam=b, ptc=False)
#from georges.plotting import *
#with plt.style.context('word'):
#    fig = plt.figure(1)

#    ax1 = fig.add_subplot(111)
#    prepare(ax1, bl)
#    aperture(ax1, bl)
    #twiss(ax1, bl_twiss)
    #track(ax1, bl, b)
